# 8. Neural Networks and Deep Learning

This week, we will learn about neural networks and how to build a model for clothing classifier.

## 8.1 Fashion classification

Get the Dataset:
- Full (~322MB): https://github.com/alexeygrigorev/clothing-dataset
- Subset (~212MB): https://github.com/alexeygrigorev/clothing-dataset-small 

Links:
- https://cs231n.github.io/

In [1]:
!git clone git@github.com:alexeygrigorev/clothing-dataset.git
!git clone git@github.com:alexeygrigorev/clothing-dataset-small.git

fatal: destination path 'clothing-dataset' already exists and is not an empty directory.
fatal: destination path 'clothing-dataset-small' already exists and is not an empty directory.


## 8.2 Tensorflow and Keras

- Installing Tensorflow
- Loading Images

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## 8.3 Pre-trained Convoluational Neural Networks (CNNs)

- Imagenet dataset: https://www.image-net.org/
- Pre-trained models: https://keras.io/api/applications/

## 8.4 Convolutional Neural Networks

- Types of layers: convolutional and dense
- Convolutional layers and filters
- Dense layers

There are more layers. Read here: https://cs231n.github.io/

## 8.5 Transfer Learning

- Reading data with `ImageDataGenerator`
- Train `Xception` on smaller images $150\times150$

(Better to run it with a GPU)

## 8.6 Adjust the learning rate

- What's the learning rate
- Trying different values

## 8.7 Checkpointing

- Saving the model only
- Training the model with callbacks

## 8.8 Adding more layers

- Adding one inner dense layer
- Experimenting with different sizes of inner layer

## 8.9 Regularization and dropout

- Regularizing by freezing a part of the network
- Adding dropout to our model
- Experimenting with different values

## 8.10 Data augmentation

- Different data augmentations
- Training a model with augmentations
- How to select data augmentations?

## 8.11 Trainingm a larger model

- Train a $299\times299$ model

## 8.12 Using the model

- Loading the model
- Evaluating the model
- Getting predictions

## 8.13 Summary

- We can use pre-trained models for general image classification
- Convolutional layers let us turn an image into a vector
- Dense layers use the vector to make the predictions
- Instead of training a model from scrath, we can use transfer learning and re-use already trained convolutional layers